# **Cuarto conjunto de tareas a realizar**

## Paquetes necesarios e inicializaciones

La siguiente práctica consta dos partes principales, la primera de ellas basada en YOLO y en detección de matrículas personas y vehículos y la segunda en OCR (Optical Character Recognition).

Si se tiene una tarjeta gráfica de NVIDIA se puede utilizar la GPU haciendo uso de CUDA, para instalar CUDAv11.6 hacer uso del siguiente script.

In [1]:
import cv2
import math
import yaml
import csv
from collections import defaultdict
import numpy as np
from ultralytics import YOLO

In [2]:
# === ENTRENAMIENTO DEL MODELO ===
print("Iniciando entrenamiento del modelo")

# Carga el modelo base (preentrenado)
model = YOLO("yolo11n.pt")

# Entrena con tu dataset y configuración
results = model.train(
    data="data.yaml",
    imgsz=416,
    batch=4,
    device="cpu",
    epochs=40,
    project="runs/train_custom",
    name="exp1",
    exist_ok=False
)

print("Entrenamiento completado.")

Iniciando entrenamiento del modelo
Ultralytics 8.3.223  Python-3.9.23 torch-2.8.0+cpu CPU (12th Gen Intel Core i7-12700H)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=4, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=40, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=416, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=exp12, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, 

KeyboardInterrupt: 

In [3]:
print("CÁLCULO MANUAL DESDE MATRIZ DE CONFUSIÓN:")

# 1. Obtén la matriz de confusión (es un array de NumPy)
matrix = results.confusion_matrix.matrix

if matrix is not None and matrix.size > 0:
    # 2. Calcula la suma de la diagonal (aciertos correctos)
    # np.diag() extrae la diagonal
    aciertos_correctos = np.diag(matrix).sum()

    # 3. Calcula el total de predicciones (suma de toda la matriz)
    total_predicciones = matrix.sum()

    # 4. Calcula el acierto
    if total_predicciones > 0:
        accuracy_manual = aciertos_correctos / total_predicciones
        
        print(f"Matriz de Confusión: \n{matrix}")
        print(f"Aciertos (Diagonal): {aciertos_correctos}")
        print(f"Total de Muestras: {total_predicciones}")
        print(f"Porcentaje de Acierto (Accuracy Manual): {accuracy_manual * 100:.2f}%")
    else:
        print("La matriz de confusión está vacía.")
else:
    print("No se generó matriz de confusión (quizás no es un modelo de detección/clasificación).")

CÁLCULO MANUAL DESDE MATRIZ DE CONFUSIÓN:


NameError: name 'results' is not defined

In [12]:
import cv2
import yaml
import math
import numpy as np
from collections import defaultdict
from ultralytics import YOLO
import csv

# === CARGAR CLASES PERSONALIZADAS ===
with open("data.yaml", "r") as f:
    data = yaml.safe_load(f)
classNames = data["names"]

print(f"Clases personalizadas: {classNames}")

# === CARGAR EL MODELO ENTRENADO ===
model = YOLO("runs/train_custom/exp1/weights/best.pt")

# === TRACKING EN VÍDEO ===
print("Iniciando detección y tracking")

video_path = "./Resources/test.mp4"
vid = cv2.VideoCapture(video_path)

if not vid.isOpened():
    print("Error: no se pudo abrir el video.")
    exit()

track_history = defaultdict(lambda: [])

# --- Definir el nuevo tamaño fijo ---
new_width = 1000
new_height = 700

# <<<<< 1. Preparar el archivo CSV para guardar los resultados
csv_filename = "tracking_results.csv"
with open(csv_filename, 'w', newline='', encoding='utf-8') as csv_file:
    csv_writer = csv.writer(csv_file)
    
    # <<<<< 2. MODIFICADO: Añadir coordenadas normalizadas al encabezado
    csv_writer.writerow(['frame', 'objeto detectado', 'confianza', 'id', 'x_center_norm', 'y_center_norm'])
    
    print(f"Guardando resultados en {csv_filename}...")

    frame_nmr = 0

    while True:
        ret, frame = vid.read()
        if not ret:
            print("Fin del video.")
            break
            
        frame_nmr += 1
        results = model.track(frame, persist=True, stream=True)

        for r in results:
            boxes = r.boxes
            for box in boxes:
                # --- Coordenadas en Píxeles (para dibujar y trazar historial) ---
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                x_center = int((x1 + x2) / 2)
                y_center = int((y1 + y2) / 2)

                # <<<<< 3. NUEVO: Obtener coordenadas normalizadas
                # box.xywhn[0] devuelve [x_centro_norm, y_centro_norm, ancho_norm, alto_norm]
                norm_coords = box.xywhn[0]
                x_center_norm = round(float(norm_coords[0]), 6) # Redondeamos a 6 decimales
                y_center_norm = round(float(norm_coords[1]), 6) # Redondeamos a 6 decimales

                # --- Resto de los datos ---
                confidence = math.ceil((box.conf[0] * 100)) / 100
                cls = int(box.cls[0])
                class_name = classNames[cls]

                track_id = ""
                if hasattr(box, "id") and box.id is not None:
                    track_id = str(int(box.id[0].tolist()))

                # <<<<< 4. MODIFICADO: Escribir coordenadas normalizadas en el CSV
                csv_writer.writerow([frame_nmr, class_name, confidence, track_id, x_center_norm, y_center_norm])

                # --- (El resto de tu código de dibujo sigue igual) ---
                
                escala = int((cls / len(classNames)) * 255 * 3)
                if escala >= 255 * 2: R, G, B = 255, 255, escala - 255 * 2
                elif escala >= 255: R, G, B = 255, escala - 255, 0
                else: R, G, B = escala, 0, 0

                cv2.rectangle(frame, (x1, y1), (x2, y2), (R, G, B), 3)
                cv2.putText(
                    frame, f"{track_id} {class_name} {confidence}", (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2
                )

                if hasattr(box, "id") and box.id is not None:
                    tid = int(box.id[0])
                    # (x_center, y_center) en píxeles se siguen usando aquí
                    track = track_history[tid]
                    track.append((x_center, y_center))
                    if len(track) > 30: track.pop(0)
                    points = np.hstack(track).astype(np.int32).reshape((-1, 1, 2))
                    cv2.polylines(frame, [points], isClosed=False, color=(230, 230, 230), thickness=2)

        
        # <<<<< AQUÍ ES DONDE SE REDIMENSIONA EL VÍDEO >>>>>
        
        # --- Opción B: Usar un tamaño fijo ---
        if frame.shape[1] > 0 and frame.shape[0] > 0:
            resized_frame = cv2.resize(frame, (new_width, new_height), interpolation=cv2.INTER_LINEAR)
        else:
            resized_frame = frame # Usar el original si hay error

        # Muestra el frame redimensionado
        cv2.imshow("YOLO Tracking - Modelo Personalizado", resized_frame)

        key = cv2.waitKey(1) & 0xFF
        if key == ord("q") or key == 27:
            print("Detección interrumpida por el usuario.")
            break

vid.release()
cv2.destroyAllWindows()
print(f"Proceso finalizado. Resultados guardados en {csv_filename}.")

Clases personalizadas: ['Green Light', 'Red Light', 'Speed Limit 10', 'Speed Limit 100', 'Speed Limit 110', 'Speed Limit 120', 'Speed Limit 20', 'Speed Limit 30', 'Speed Limit 40', 'Speed Limit 50', 'Speed Limit 60', 'Speed Limit 70', 'Speed Limit 80', 'Speed Limit 90', 'Stop']
Iniciando detección y tracking
Guardando resultados en tracking_results.csv...

0: 416x416 1 Stop, 136.8ms
Speed: 13.1ms preprocess, 136.8ms inference, 1.0ms postprocess per image at shape (1, 3, 416, 416)

0: 416x416 1 Stop, 161.5ms
Speed: 1.8ms preprocess, 161.5ms inference, 1.9ms postprocess per image at shape (1, 3, 416, 416)

0: 416x416 1 Stop, 131.4ms
Speed: 2.8ms preprocess, 131.4ms inference, 1.1ms postprocess per image at shape (1, 3, 416, 416)

0: 416x416 (no detections), 105.9ms
Speed: 1.4ms preprocess, 105.9ms inference, 1.0ms postprocess per image at shape (1, 3, 416, 416)

0: 416x416 1 Stop, 130.6ms
Speed: 2.1ms preprocess, 130.6ms inference, 1.3ms postprocess per image at shape (1, 3, 416, 416)

0